# Toronto Neighbourhood Clustering Assignment

Data web scraped from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.

## Scraping Section

In [1]:
!pip install lxml html5lib beautifulsoup4

Read wiki page using pandas html table tag

In [3]:
! pip install pandas
import pandas as pd

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)

print(len(dfs))

  Using cached pytz-2020.1-py2.py3-none-any.whl (510 kB)
3


In [4]:
print(dfs[0])

    Postal Code           Borough  \
0           M1A      Not assigned   
1           M2A      Not assigned   
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
..          ...               ...   
175         M5Z      Not assigned   
176         M6Z      Not assigned   
177         M7Z      Not assigned   
178         M8Z         Etobicoke   
179         M9Z      Not assigned   

                                          Neighborhood  
0                                         Not assigned  
1                                         Not assigned  
2                                            Parkwoods  
3                                     Victoria Village  
4                            Regent Park, Harbourfront  
..                                                 ...  
175                                       Not assigned  
176                                       Not assigned  
177                                       

Remove any rows without a borough assigned

In [7]:
table = dfs[0] 
indices = table[table['Borough'] =='Not assigned'].index # only want rows without "Not assigned" in boroughs

table.drop(indices , inplace=True)
table.head(15)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [11]:
table.loc[table['Neighborhood'] =='Not assigned' , 'Neighborhood'] = table['Borough']
table.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


On the new wiki page, postal codes already only appear once. Let's check M5A

In [17]:
print(table.loc[table['Postal Code'] == 'M5A', 'Neighborhood'])

4    Regent Park, Harbourfront
Name: Neighborhood, dtype: object


In [12]:
table.shape

(103, 3)